In [1]:
from keras.models import Sequential
from keras.layers import Dense

Using Theano backend.


In [2]:
import numpy as np
np.random.seed(7)
import pypyodbc
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import mpld3
mpld3.enable_notebook()
#import matplotlib
#matplotlib.use('nbagg')
from sklearn.preprocessing import Imputer
from pandas_ml import ConfusionMatrix
from sklearn import preprocessing
import pandas as pd

#### Read the training dataset

In [3]:
train_df = pd.read_csv('blood_train.csv')
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 6 columns):
Unnamed: 0                     576 non-null int64
Months since Last Donation     576 non-null int64
Number of Donations            576 non-null int64
Total Volume Donated (c.c.)    576 non-null int64
Months since First Donation    576 non-null int64
Made Donation in March 2007    576 non-null int64
dtypes: int64(6)
memory usage: 27.1 KB


#### Preprocess the data

In [4]:
train_df=train_df.rename(columns = {'Unnamed: 0':'ID'})
train_df['donation_span'] = train_df['Months since First Donation'] - train_df['Months since Last Donation']

train_df['cc per year'] = train_df['donation_span'] / 12
train_df['cc per year'] = train_df['Total Volume Donated (c.c.)'] / train_df['cc per year']
train_df['cc per year'] = train_df['cc per year'].replace([np.inf, -np.inf], np.nan)
train_df['cc per year'] = train_df['cc per year'].fillna(0)
train_df['Avg #Donation per year'] = train_df['Months since First Donation'] / 12
train_df['Avg #Donation per year'] = train_df['Number of Donations'] / train_df['Avg #Donation per year'] 
train_df['Avg #Donation per year'] = train_df['Avg #Donation per year'].replace([np.inf, -np.inf], np.nan)
train_df['Avg #Donation per year'] = train_df['Avg #Donation per year'].fillna(0)
train_df['donates evry x months'] = train_df['donation_span'] / train_df['Number of Donations']
train_df['first_time'] = train_df['donation_span'].apply(lambda x : 1 if x == 0 else 0)

In [5]:
train_df['donation_span_yr'] = train_df['donation_span'] / 12

In [6]:
train_df

,ID,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,Made Donation in March 2007,donation_span,cc per year,Avg #Donation per year,donates evry x months,first_time,donation_span_yr
0,619,2,50,12500,98,1,96,1562.500000,6.122449,1.920000,0,8.000000
1,664,0,13,3250,28,1,28,1392.857143,5.571429,2.153846,0,2.333333
2,441,1,16,4000,35,1,34,1411.764706,5.485714,2.125000,0,2.833333
3,160,2,20,5000,45,1,43,1395.348837,5.333333,2.150000,0,3.583333
4,358,1,24,6000,77,0,76,947.368421,3.740260,3.166667,0,6.333333
5,335,4,4,1000,4,0,0,0.000000,12.000000,0.000000,1,0.000000
6,47,2,7,1750,14,1,12,1750.000000,6.000000,1.714286,0,1.000000
7,164,1,12,3000,35,0,34,1058.823529,4.114286,2.833333,0,2.833333
8,736,5,46,11500,98,1,93,1483.870968,5.632653,2.021739,0,7.750000
9,436,0,3,750,4,0,4,2250.000000,9.000000,1.333333,0,0.333333


#### Select all the numerical columns as feature for our deep learning model

In [10]:
ftcol = train_df.columns.difference(['ID','Made Donation in March 2007'])
len(ftcol)

10

In [11]:
from sklearn.model_selection import train_test_split

#### Split the training data for validation 

In [12]:
train,test = train_test_split(train_df,test_size=0.3)
len(train)

403

In [13]:
#ftcol = ['donation_span_yr','donates evry x months','Total Volume Donated (c.c.)','Avg #Donation per year','cc per year','Months since Last Donation','Months since First Donation']
features = train[ftcol].values
label = train['Made Donation in March 2007'].values

In [1]:
### initial model for validation

In [14]:
model = Sequential()
model.add(Dense(10, input_dim=10, init='normal', activation='relu'))
model.add(Dense(5, init='normal', activation='relu'))
model.add(Dense(1, init='normal', activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.fit(features,label,nb_epoch=50, batch_size = 50,verbose=1)

Epoch 1/50
403/403 [==============================] - 0s - loss: 0.6667 - acc: 0.7519     
Epoch 2/50
403/403 [==============================] - 0s - loss: 0.6609 - acc: 0.7519     
Epoch 3/50
403/403 [==============================] - 0s - loss: 0.6578 - acc: 0.7519     
Epoch 4/50
403/403 [==============================] - 0s - loss: 0.6552 - acc: 0.7519     
Epoch 5/50
403/403 [==============================] - 0s - loss: 0.6534 - acc: 0.7519     
Epoch 6/50
403/403 [==============================] - 0s - loss: 0.6502 - acc: 0.7519     
Epoch 7/50
403/403 [==============================] - 0s - loss: 0.6480 - acc: 0.7543     
Epoch 8/50
403/403 [==============================] - 0s - loss: 0.6463 - acc: 0.7568     
Epoch 9/50
403/403 [==============================] - 0s - loss: 0.6449 - acc: 0.7568     
Epoch 10/50
403/403 [==============================] - 0s - loss: 0.6435 - acc: 0.7568     
Epoch 11/50
403/403 [==============================] - 0s - loss: 0.6396 - acc: 0.7593   

In [16]:
ypred = model.predict(test[ftcol].values,batch_size=30,verbose=1)

 30/173 [====>.........................] - ETA: 0s

In [17]:
test['keraspred'] = ypred

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [18]:
claspred = model.predict_classes(test[ftcol].values,batch_size=30,verbose=1)

 30/173 [====>.........................] - ETA: 0s

In [19]:
test['clp'] = claspred

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [2]:
### Confusion matrix for validation

In [20]:
cm = ConfusionMatrix(test['Made Donation in March 2007'],test['clp'])
cm

Predicted    0  1  __all__
Actual                    
0          134  1      135
1           36  2       38
__all__    170  3      173

In [21]:
cm.print_stats()

population: 173
P: 38
N: 135
PositiveTest: 3
NegativeTest: 170
TP: 2
TN: 134
FP: 1
FN: 36
TPR: 0.0526315789474
TNR: 0.992592592593
PPV: 0.666666666667
NPV: 0.788235294118
FPR: 0.00740740740741
FDR: 0.333333333333
FNR: 0.947368421053
ACC: 0.78612716763
F1_score: 0.0975609756098
MCC: 0.143431392339
informedness: 0.04522417154
markedness: 0.454901960784
prevalence: 0.219653179191
LRP: 7.10526315789
LRN: 0.954438334643
DOR: 7.44444444444
FOR: 0.211764705882


##### Bring in testing data for making final predictions

In [30]:
test.to_csv('blodkeras.csv',index=False)

In [22]:
test_df = pd.read_csv('blood_test.csv')

In [23]:
test_df=test_df.rename(columns = {'Unnamed: 0':'ID'})
test_df['donation_span'] = test_df['Months since First Donation'] - train_df['Months since Last Donation']

test_df['cc per year'] = test_df['donation_span'] / 12
test_df['cc per year'] = test_df['Total Volume Donated (c.c.)'] / test_df['cc per year']
test_df['cc per year'] = test_df['cc per year'].replace([np.inf, -np.inf], np.nan)
test_df['cc per year'] = test_df['cc per year'].fillna(0)
test_df['Avg #Donation per year'] = test_df['Months since First Donation'] / 12
test_df['Avg #Donation per year'] = test_df['Number of Donations'] / test_df['Avg #Donation per year'] 
test_df['Avg #Donation per year'] = test_df['Avg #Donation per year'].replace([np.inf, -np.inf], np.nan)
test_df['Avg #Donation per year'] = test_df['Avg #Donation per year'].fillna(0)
test_df['donates evry x months'] = test_df['donation_span'] / test_df['Number of Donations']
test_df['first_time'] = test_df['donation_span'].apply(lambda x : 1 if x == 0 else 0)

In [24]:
test_df['donation_span_yr'] = test_df['donation_span'] / 12

In [25]:
test_df

,ID,Months since Last Donation,Number of Donations,Total Volume Donated (c.c.),Months since First Donation,donation_span,cc per year,Avg #Donation per year,donates evry x months,first_time,donation_span_yr
0,659,2,12,3000,52,50.0,720.000000,2.769231,4.166667,0,4.166667
1,276,21,7,1750,38,38.0,552.631579,2.210526,5.428571,0,3.166667
2,263,4,1,250,4,3.0,1000.000000,3.000000,3.000000,0,0.250000
3,303,11,11,2750,38,36.0,916.666667,3.473684,3.272727,0,3.000000
4,83,4,12,3000,34,33.0,1090.909091,4.235294,2.750000,0,2.750000
5,500,3,21,5250,42,38.0,1657.894737,6.000000,1.809524,0,3.166667
6,530,4,2,500,4,2.0,3000.000000,6.000000,1.000000,0,0.166667
7,244,14,1,250,14,13.0,230.769231,0.857143,13.000000,0,1.083333
8,249,23,2,500,87,82.0,73.170732,0.275862,41.000000,0,6.833333
9,728,14,4,1000,64,64.0,187.500000,0.750000,16.000000,0,5.333333


In [26]:
model.fit(train_df[ftcol].values,train_df['Made Donation in March 2007'].values,nb_epoch=100, batch_size = 50,verbose=1)

Epoch 1/100
576/576 [==============================] - 0s - loss: 0.5231 - acc: 0.7691     
Epoch 2/100
576/576 [==============================] - 0s - loss: 0.5197 - acc: 0.7708     
Epoch 3/100
576/576 [==============================] - 0s - loss: 0.5152 - acc: 0.7743     
Epoch 4/100
576/576 [==============================] - 0s - loss: 0.5119 - acc: 0.7708     
Epoch 5/100
576/576 [==============================] - 0s - loss: 0.5091 - acc: 0.7708     
Epoch 6/100
576/576 [==============================] - 0s - loss: 0.5056 - acc: 0.7674     
Epoch 7/100
576/576 [==============================] - 0s - loss: 0.5031 - acc: 0.7691     
Epoch 8/100
576/576 [==============================] - 0s - loss: 0.5047 - acc: 0.7708     
Epoch 9/100
576/576 [==============================] - 0s - loss: 0.5001 - acc: 0.7656     
Epoch 10/100
576/576 [==============================] - 0s - loss: 0.5001 - acc: 0.7674     
Epoch 11/100
576/576 [==============================] - 0s - loss: 0.4985 - acc

###### make the predictions and store to a file

In [27]:
ypred = model.predict(test_df[ftcol].values,batch_size=30,verbose=1)
test_df['Made Donation in March 2007'] = ypred
test_df.to_csv('blodkeras.csv',index=False)

 30/200 [===>..........................] - ETA: 0s